In [120]:
import mxnet as mx
mx.__version__

'1.3.0'

In [121]:
RAW_DATA_LOCATION = '../data/raw/35.txt'

In [160]:
with open(RAW_DATA_LOCATION) as f:
    raw_data = f.read()
raw_data = raw_data[44332: -24182]
print('\nBEGINNING\n', raw_data[:1000])
print( '\nEND\n', raw_data[-1000:])
#raw_data_val = raw_data[-len(raw_data)//3:]
#raw_data = raw_data[:2*len(raw_data)//3]
#print(len(raw_data), len(raw_data_val), len(raw_data[:2*len(raw_data)//3]))
#print( '\n\n\n\n\n\n', len(raw_data), len(raw_data_val), '\n\n\n\n\n\n')
#print(raw_data[-100:], '\n\n\n\n\n\n' + raw_data_val[-100:])

with open('../data/raw/train_data.txt', 'w+') as output_file:
    output_file.write(raw_data)
    
#with open('../data/raw/test_data.txt', 'w+') as output_file:
 #   output_file.write(raw_data_val)



BEGINNING
 The Time Machine, by H. G. Wells [1898]



I


The Time Traveller (for so it will be convenient to speak of him)
was expounding a recondite matter to us. His grey eyes shone and
twinkled, and his usually pale face was flushed and animated. The
fire burned brightly, and the soft radiance of the incandescent
lights in the lilies of silver caught the bubbles that flashed and
passed in our glasses. Our chairs, being his patents, embraced and
caressed us rather than submitted to be sat upon, and there was that
luxurious after-dinner atmosphere when thought roams gracefully
free of the trammels of precision. And he put it to us in this
way--marking the points with a lean forefinger--as we sat and lazily
admired his earnestness over this new paradox (as we thought it)
and his fecundity.

'You must follow me carefully. I shall have to controvert one or two
ideas that are almost universally accepted. The geometry, for
instance, they taught you at school is founded on a misconception

In [161]:
import gluonnlp as nlp
import multiprocessing as mp
import itertools
import numpy as np
from mxnet import gluon, autograd

In [ ]:
@register(segment=['train', 'val', 'test'])
class TimeMachineDataSet(_WikiText):
    """WikiText-2 word-level dataset for language modeling, from Salesforce research.

    From
    https://einstein.ai/research/the-wikitext-long-term-dependency-language-modeling-dataset

    License: Creative Commons Attribution-ShareAlike

    Parameters
    ----------
    segment : {'train', 'val', 'test'}, default 'train'
        Dataset segment.
    skip_empty : bool, default True
        Whether to skip the empty samples produced from sample_splitters. If False, `bos` and `eos`
        will be added in empty samples.
    tokenizer : function, default str.split
        A function that splits each sample string into list of tokens.
    bos : str or None, default None
        The token to add at the begining of each sentence. If None, nothing is added.
    eos : str or None, default '<eos>'
        The token to add at the end of each sentence. If None, nothing is added.
    root : str, default '$MXNET_HOME/datasets/wikitext-2'
        Path to temp folder for storing data.
        MXNET_HOME defaults to '~/.mxnet'.
    """

    def __init__(self, segment='train', skip_empty=True,
                 tokenizer=lambda s: s.split(),
                 bos=None, eos=C.EOS_TOKEN, root=os.path.join(
                     _get_home_dir(), 'datasets', 'wikitext-2'), **kwargs):
        self._archive_file = ('wikitext-2-v1.zip', '3c914d17d80b1459be871a5039ac23e752a53cbe')
        self._data_file = {'train': ('wiki.train.tokens',
                                     '863f29c46ef9d167fff4940ec821195882fe29d1'),
                           'val': ('wiki.valid.tokens',
                                   '0418625c8b4da6e4b5c7a0b9e78d4ae8f7ee5422'),
                           'test': ('wiki.test.tokens',
                                    'c7b8ce0aa086fb34dab808c5c49224211eb2b172')}
        super(WikiText2,
              self).__init__('wikitext-2', segment, bos, eos, skip_empty, root,
                             tokenizer=tokenizer, **kwargs)

In [162]:
train_dataset = nlp.data.LanguageModelDataset('../data/raw/train_data.txt')
train_data_iter = gluon.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

In [167]:
train_dataset[0]

['The',
 'Time',
 'Machine,',
 'by',
 'H.',
 'G.',
 'Wells',
 '[1898]',
 'I',
 'The',
 'Time',
 'Traveller',
 '(for',
 'so',
 'it',
 'will',
 'be',
 'convenient',
 'to',
 'speak',
 'of',
 'him)',
 'was',
 'expounding',
 'a',
 'recondite',
 'matter',
 'to',
 'us.',
 'His',
 'grey',
 'eyes',
 'shone',
 'and',
 'twinkled,',
 'and',
 'his',
 'usually',
 'pale',
 'face',
 'was',
 'flushed',
 'and',
 'animated.',
 'The',
 'fire',
 'burned',
 'brightly,',
 'and',
 'the',
 'soft',
 'radiance',
 'of',
 'the',
 'incandescent',
 'lights',
 'in',
 'the',
 'lilies',
 'of',
 'silver',
 'caught',
 'the',
 'bubbles',
 'that',
 'flashed',
 'and',
 'passed',
 'in',
 'our',
 'glasses.',
 'Our',
 'chairs,',
 'being',
 'his',
 'patents,',
 'embraced',
 'and',
 'caressed',
 'us',
 'rather',
 'than',
 'submitted',
 'to',
 'be',
 'sat',
 'upon,',
 'and',
 'there',
 'was',
 'that',
 'luxurious',
 'after-dinner',
 'atmosphere',
 'when',
 'thought',
 'roams',
 'gracefully',
 'free',
 'of',
 'the',
 'trammels',
 

In [174]:
num_gpus = 1
context = [mx.gpu(i) for i in range(num_gpus)] if num_gpus else [mx.cpu()]
log_interval = 200

batch_size = 20 * len(context)
lr = 20
epochs = 3
bptt = 35
grad_clip = 0.25


dataset_name = 'wikitext-2'
train_dataset, val_dataset, test_dataset = [nlp.data.WikiText2(segment=segment, 
                                                               root='../data/', 
                                                               bos=None, eos='<eos>',
                                                               skip_empty=False)
                                            for segment in ['train', 'val', 'test']]

vocab = nlp.Vocab(nlp.data.Counter(train_dataset[0]), padding_token=None, bos_token=None)

train_data, val_data, test_data = [x.bptt_batchify(vocab, bptt, batch_size,
                                                   last_batch='discard')
                                   for x in [train_dataset, val_dataset, test_dataset]]